In [1]:
import pymongo
import re
import pprint
import json
import jieba
from collections import Counter
import time
import math
import logging
from gensim.models import word2vec
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

D:\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


# 將資料由pymongo讀取近來

In [2]:
#登入pymongo
clinet=pymongo.MongoClient("mongodb://10.120.27.23:27017/")

In [3]:
#確認裡面有哪幾個database
clinet.database_names()

['admin',
 'iii-2017-07',
 'local',
 'mydb',
 'project',
 'rawData',
 'test',
 'test-database']

In [4]:
#選擇rawData這個db
db=clinet['rawData']

In [5]:
#確認裡面有哪幾個collection
db.collection_names()

['mobile01_tainan',
 'Tainan_final',
 'merge_tainanUni',
 'Tbike',
 'ptt_tainan_address',
 'AipinwanFood',
 'ptt_tainan_tel',
 'Tainan_final_name_id_1',
 'pixnet_tainan_author',
 'FinishTainanBigTable',
 'ptt_tainan']

In [6]:
#將愛評網讀入
ipeenjson=list(db.Tainan_final.find({}))

In [7]:
ipeenjson[0]

{'_id': ObjectId('59afa8e6f3a322258443a5e3'),
 'address': '台南市安南區安北路433號',
 'average_consumption': '42',
 'business_hours': '09:00~23:00',
 'comment': [{'click': 1004,
   'content': '簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji

# 將結巴字典加入自己的許多詞

In [8]:

#結巴 匯入自己的詞典
jieba.set_dictionary('D:/WordLibrary/JiebaUse/jieba_dict.txt.big')

#停止詞字典stopwordset
stopwordset = set()
with open('D:/WordLibrary/JiebaUse/stopwords.txt', 'r', encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
        
#店家名稱字典dienlist
dienlist = [dien["name"] for dien in ipeenjson]

#店家種類字典stylelist
stylelist = [dien["style"] for dien in ipeenjson]

#店家種類字典introductionlist
introductionlist = [dien["introduction"] for dien in ipeenjson]

#店家種類字典coordinatelist
coordinatelist = [dien['coordinate'] for dien in ipeenjson]

#自製餐廳相關字字典pinlist
with open("D:\Data\csvtest\MyDict.csv") as f:
    mydict=f.read()
#linewords pindict每個字的評分dict
linewords=[{myword:float(line.split(",")[0]) for myword in line.split(",")[1:]}\
           for line in mydict.split("\n") if line.split(",")[0]!=""]
pindict={}
for word in linewords:
    pindict.update(word)
pinlist=list(pindict.keys())

#將以上字典都加入斷詞行列
for i in dienlist:
    jieba.add_word(i)
for i in stylelist:
    jieba.add_word(i)
for i in pinlist:
    if i !="":
        jieba.add_word(i)
        
#店家評論總數Ncommentlist
Ncommentlist = [len(i["comment"]) for i in ipeenjson]


    

Building prefix dict from D:\WordLibrary\JiebaUse\jieba_dict.txt.big ...
Loading model from cache C:\Users\Java\AppData\Local\Temp\jieba.uf3aabe1eaf7c86dda8313189b0c5c683.cache
Loading model cost 1.240 seconds.
Prefix dict has been built succesfully.


In [9]:
#各店家評論diencommentlist
diencommentlist=[[j["content"] for j in i["comment"]] for i in ipeenjson]
diencommentlist[0]

['簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/',
 '來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜',
 '這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是

In [10]:
#各店家回覆dienmessagelist
dienmessagelist=[[''.join(j["message"]) for j in i["comment"] if j["message"]!=None and j["message"]!='null'] for i in ipeenjson]
dienmessagelist[0]

['感謝分享，也幫我推一個，謝謝http://www.ipeen.com.tw/comment/943950',
 '到台南好像不能錯過安平的豆花耶給你非常實用也希望你能給我實用喔【電影類別 / 動作】玩命關頭7-復仇之路 回歸到起點推~~~同是台南人XD這家真的好吃http://www.ipeen.com.tw/comment/838440麻煩支持 一下喔:)',
 '鍋燒意麵這款中山店沒有賣!!!!',
 '竹炭豆畫...黑色的嘛==?感覺鮮奶的比較好吃去安平，必吃美食~~讚！',
 '好像我去過這家！很面熟，但是不太確定。',
 '回推囉，豆花看起來不錯呢！',
 '感覺你們玩超多天的耶XD呵呵 我跟阿左去三天兩夜都感覺沒有你們玩得多呢我喜歡粉圓豆花。這一家商妮去台南玩的時候也有去吃，人好多~',
 '豆花看起來好好吃><紅豆在上面顏色就看起來很漂亮最近的美食分享，喜歡的話幫忙給個讚吧非常謝謝您GODIVA比利時巧克力●奢華享受http://www.ipeen.com.tw/comment/611570IOU Cafe●甜點美饌 http://www.ipeen.com.tw/comment/611714安平豆花我也超愛~下次再來台南~安平豆花地鍋燒意麵和對面的福記水煎包也都蠻好吃的~~很特別的豆花.看起來很嫩那天下去看到好多人，就沒有去排了!!!',
 '每次去安平 我都必吃那家豆花耶~~~超愛!!',
 '我喜歡粉圓加檸檬，酸酸甜甜又清涼上次去台南很猶豫要吃這間還是黑豆花，結果選了黑豆花太太太銷魂了謝謝分享',
 '超愛珍珠豆花的啦~~~~~~檸檬也蠻好吃的~隔壁鍋燒意麵也好吃',
 '好久沒吃了! 上次很像點珍珠檸檬豆花~好吃也喜歡酸甜感我上次也是點檸檬的耶!  紅豆看起來挺不賴的這不是排隊排慘的那家嗎XD豆花好綿啊~看起來就很好吃可惜我上回去~找不到這家，最後隨便路邊找了一間進去吃，結果難吃到爆~~~我也超愛這家豆花，只要有到安平我都會來這家，記的以前他們還沒出名時，我就常去那吃，當時是家小小的店，店裡桌以才3~4張而已呢檸檬的超好吃!!!他們家的鍋燒意面也很不錯~下次可以試試~!\t※ 引述 ✿花露露✿ 之言：\t檸檬的超好吃!!!\t他們家的鍋燒意面也很不錯~下次可以試試~!蝦密??還有賣鍋燒意麵阿?真的假的去很多次了耶~都沒發現....XD\t※

In [11]:
#回覆數量
Nmessagelist=[len(dienmessage) for dienmessage in dienmessagelist]

In [12]:
#全部回覆+全部評論
dienallpinlist=[diencomment+dienmessage for diencomment,dienmessage in zip(diencommentlist,dienmessagelist)]
dienallpinlist[0]


['簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/',
 '來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜',
 '這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是

In [13]:
# for megs in dienmessagelist:
#     for meg in megs:
#         if meg!=None and meg!='null'and meg!='':
#             print(" ".join(meg))

#把所有回覆合併
messagecombinlist=[" ".join(["".join(meg) for meg in megs if meg!=None and meg!='null'and meg!='']) for megs in dienmessagelist]
messagecombinlist[0]

'感謝分享，也幫我推一個，謝謝http://www.ipeen.com.tw/comment/943950 到台南好像不能錯過安平的豆花耶給你非常實用也希望你能給我實用喔【電影類別 / 動作】玩命關頭7-復仇之路 回歸到起點推~~~同是台南人XD這家真的好吃http://www.ipeen.com.tw/comment/838440麻煩支持 一下喔:) 鍋燒意麵這款中山店沒有賣!!!! 竹炭豆畫...黑色的嘛==?感覺鮮奶的比較好吃去安平，必吃美食~~讚！ 好像我去過這家！很面熟，但是不太確定。 回推囉，豆花看起來不錯呢！ 感覺你們玩超多天的耶XD呵呵 我跟阿左去三天兩夜都感覺沒有你們玩得多呢我喜歡粉圓豆花。這一家商妮去台南玩的時候也有去吃，人好多~ 豆花看起來好好吃><紅豆在上面顏色就看起來很漂亮最近的美食分享，喜歡的話幫忙給個讚吧非常謝謝您GODIVA比利時巧克力●奢華享受http://www.ipeen.com.tw/comment/611570IOU Cafe●甜點美饌 http://www.ipeen.com.tw/comment/611714安平豆花我也超愛~下次再來台南~安平豆花地鍋燒意麵和對面的福記水煎包也都蠻好吃的~~很特別的豆花.看起來很嫩那天下去看到好多人，就沒有去排了!!! 每次去安平 我都必吃那家豆花耶~~~超愛!! 我喜歡粉圓加檸檬，酸酸甜甜又清涼上次去台南很猶豫要吃這間還是黑豆花，結果選了黑豆花太太太銷魂了謝謝分享 超愛珍珠豆花的啦~~~~~~檸檬也蠻好吃的~隔壁鍋燒意麵也好吃 好久沒吃了! 上次很像點珍珠檸檬豆花~好吃也喜歡酸甜感我上次也是點檸檬的耶!  紅豆看起來挺不賴的這不是排隊排慘的那家嗎XD豆花好綿啊~看起來就很好吃可惜我上回去~找不到這家，最後隨便路邊找了一間進去吃，結果難吃到爆~~~我也超愛這家豆花，只要有到安平我都會來這家，記的以前他們還沒出名時，我就常去那吃，當時是家小小的店，店裡桌以才3~4張而已呢檸檬的超好吃!!!他們家的鍋燒意面也很不錯~下次可以試試~!\t※ 引述 ✿花露露✿ 之言：\t檸檬的超好吃!!!\t他們家的鍋燒意面也很不錯~下次可以試試~!蝦密??還有賣鍋燒意麵阿?真的假的去很多次了耶~都沒發現....XD\t※ 引述 品味廣東粥 之言：\t我也超愛這家豆花，只要有到安平我都會來這家，記的以前他們還沒

In [14]:
#把所有評論合併
commentcombinelist = [" ".join([j["content"] for j in i["comment"]]) for i in ipeenjson]
commentcombinelist[0]

'簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/ 來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜 這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是平日的暑假店裡還是

In [15]:
#合併評論+回覆
pincombinelist=[comment+" "+message for comment,message in zip(commentcombinelist,messagecombinlist)]
pincombinelist[0]

'簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/ 來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜 這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是平日的暑假店裡還是

In [16]:
#清一下記憶體
import gc
gc.collect()

0


# 將許多可能會用來分析的項目做成list

# 做TD-IDF分析

In [17]:

st=time.time()
#取得各店家的斷完詞評論pincutlist
pincutlist = []
cutcount=0
for pincombine in pincombinelist:
    cutcount+=1
    if cutcount%200==0:
        print("以切"+str(cutcount)+"篇文章的詞")
    words = jieba.cut(pincombine, cut_all=False)
    pincut=" ".join([word for word in words if word not in stopwordset and
                     '\u4e00' <= word and word <= '\u9fff' ])#and word not in dienlist and word not in pinlist
    pincutlist.append(pincut)
    
#所有店家的用詞加總前三百wordCountlist300
wordCountlist=[Counter(pincut.split(" ")).most_common(300) for pincut in pincutlist]

#TF計算 (TF=這個字出現的次數/這篇文章的總字數)
#(sum(dict(wordCount).values())=>一篇所有字的字數  wordkv[1]>>該文字字數  wordkv[0]>>該文字)
#TFList每篇文章的詞 的詞頻
TFList=[[((wordkv[0],wordkv[1]/sum(dict(wordCount).values()))) for wordkv in wordCount] for wordCount in wordCountlist]


#全文章共用了幾個詞(set可以去除重複，可迭代)
wordlist=[]
for wordCount in wordCountlist:
    for word in wordCount:
        wordlist.append(word[0])
wordlist=set(wordlist)

#nDien所有的文章(店家)數
ndien=len(dienlist)

#wordappear計算每個字出現於 (幾篇) 文章
wordappear={}
appearcount=0
for word in wordlist:
    appearcount+=1
    if appearcount%1000==0:
        print("以計算"+str(appearcount)+"個文字的出現篇數")
    n=0
    for pincut in pincutlist:
        if word in pincut:
            n+=1
    wordappear[word]=n
    
#IDF(逆向檔案頻率)=某詞  所有文章數/在幾篇文章出現 開log10 >>次方數為所
IDFlist={word:math.log(ndien/wordappear[word],10) for word in wordappear}

#dienTF每家店的全字TF值 wordTF每個詞的TF   wordTF[0]詞 wordTF[1]詞的TF值  IDFlist[wordTF[0]]詞的IDF值
TF_IDFlist=[Counter({wordTF[0]:wordTF[1]*IDFlist[wordTF[0]]for wordTF in dienTF})for dienTF in TFList]
for name,TFIDF in zip(dienlist,TF_IDFlist):
    if name in TFIDF:
        del TFIDF[name]


et=time.time()
print(et-st)

以切200篇文章的詞
以切400篇文章的詞
以切600篇文章的詞
以切800篇文章的詞
以切1000篇文章的詞
以切1200篇文章的詞
以切1400篇文章的詞
以切1600篇文章的詞
以切1800篇文章的詞
以切2000篇文章的詞
以切2200篇文章的詞
以切2400篇文章的詞
以切2600篇文章的詞
以切2800篇文章的詞
以切3000篇文章的詞
以切3200篇文章的詞
以切3400篇文章的詞
以切3600篇文章的詞
以切3800篇文章的詞
以切4000篇文章的詞
以切4200篇文章的詞
以切4400篇文章的詞
以切4600篇文章的詞
以切4800篇文章的詞
以切5000篇文章的詞
以切5200篇文章的詞
以切5400篇文章的詞
以切5600篇文章的詞
以計算1000個文字的出現篇數
以計算2000個文字的出現篇數
以計算3000個文字的出現篇數
以計算4000個文字的出現篇數
以計算5000個文字的出現篇數
以計算6000個文字的出現篇數
以計算7000個文字的出現篇數
以計算8000個文字的出現篇數
以計算9000個文字的出現篇數
以計算10000個文字的出現篇數
以計算11000個文字的出現篇數
以計算12000個文字的出現篇數
以計算13000個文字的出現篇數
以計算14000個文字的出現篇數
以計算15000個文字的出現篇數
以計算16000個文字的出現篇數
以計算17000個文字的出現篇數
以計算18000個文字的出現篇數
以計算19000個文字的出現篇數
以計算20000個文字的出現篇數
以計算21000個文字的出現篇數
以計算22000個文字的出現篇數
以計算23000個文字的出現篇數
以計算24000個文字的出現篇數
以計算25000個文字的出現篇數
以計算26000個文字的出現篇數
以計算27000個文字的出現篇數
以計算28000個文字的出現篇數
以計算29000個文字的出現篇數
以計算30000個文字的出現篇數
以計算31000個文字的出現篇數
以計算32000個文字的出現篇數
以計算33000個文字的出現篇數
以計算34000個文字的出現篇數
以計算35000個文字的出現篇數
以計算36000個文字的出現篇數
以計算37000個文字的出現篇數
以計算38000個文字的出現篇數
以計算39000個文字的出現篇數
以計算40000個文字的出現

# 將上面的斷詞文章combine在一起 做word2vec model

In [ ]:


#將所有的斷詞文章combine在一起，變成超大斷詞字串，並存起來
contentcutcombine=" ".join(pincutlist)
with open('D:\Data\word2VecData\TainanContent.txt','w',encoding="utf-8") as w:
    w.write(contentcutcombine)
    
#將存起來的文章用word2vec.Text8Corpus切成有順序的詞彙list(sentences)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.Text8Corpus("D:/Data/word2VecData/TainanContent.txt")

#將有順序的詞彙list丟給word2vec做訓練
model = word2vec.Word2Vec(sentences, size=300,window=6, min_count=4, workers=4,sg=1)
# sentences:當然了，這是要訓練的句子集，沒有他就不用跑了
# size:這表示的是訓練出的詞向量會有幾維
# alpha:機器學習中的學習率，這東西會逐漸收斂到 min_alpha
# sg:sg=1表示採用skip-gram(由本字去測他字 擴散),sg=0表示採用cbow(由他字測本字 集中)
# window:能往左往右看幾個字的意思
# workers:執行緒數目，除非電腦不錯，不然建議別超過 4
# min_count:若這個詞出現的次數小於min_count，那他就不會被視為訓練對象

# Save our model.
model.save("D:/Data/word2VecData/Tainan.model.bin")


In [18]:
gc.collect()

0

# 試玩word2vec的model  需解除#

In [ ]:
# from gensim.models import word2vec
# from gensim import models
# import logging

# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# model = models.Word2Vec.load('D:/Data/word2VecData/Tainan.model.bin')

# print("提供 3 種測試模式\n")
# print("輸入一個詞，則去尋找前一百個該詞的相似詞")
# print("輸入兩個詞，則去計算兩個詞的餘弦相似度")
# print("輸入三個詞，進行類比推理")

# while True:
#     try:
#         query = input()
#         q_list = query.split()

#         if len(q_list) == 1:
#             print("相似詞前 30 排序")
#             res = model.most_similar(q_list[0],topn = 30)
#             for item in res:
#                 print(item[0]+","+str(item[1]))

#         elif len(q_list) == 2:
#             print("計算 Cosine 相似度")
#             res = model.similarity(q_list[0],q_list[1])
#             print(res)
#         else:
#             print("%s之於%s，如%s之於" % (q_list[0],q_list[2],q_list[1]))
#             res = model.most_similar([q_list[0],q_list[1]], [q_list[2]], topn= 20)
#             for item in res:
#                 print(item[0]+","+str(item[1]))
#         print("----------------------------")
#     except Exception as e:
#         print(repr(e))


# 試Rocchio、bayes、KNN、SVM分類 (仿白話大數據寫法)

In [ ]:
#取得各店家的斷完詞評論contentcutlist
#店家種類字典stylelist
# from sklearn.datasets import fetch_20newsgroups
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer

#斷詞function
def cut_text_list(textlist):
    textcutlist=[]
    for text in textlist:
        cutwords=jieba.cut(text)
        cuttext=" ".join([word for word in cutwords if word not in stopwordset and '\u4e00' <= word <= '\u9fff'])
        textcutlist.append(cuttext)
    return textcutlist

#文字轉向量矩陣
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(pincutlist)
print('文字轉向量矩陣')

#向量矩陣做tfidf

tfidf_transformer=TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('tfidf')



#ptt測試集|
with open(r'D:\Data\testData\foodptt.json') as f:
    testdatas=json.load(f)
docs_new=cut_text_list(testdatas)
X_new_counts= count_vect.transform(docs_new)
x_new_tfidf=tfidf_transformer.transform(X_new_counts)

#試用Roccio分群
from sklearn.neighbors.nearest_centroid import NearestCentroid
st=time.time()
clf = NearestCentroid().fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("Roccio")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)


#試用bayes分群
from sklearn.naive_bayes import MultinomialNB
st=time.time()
clf = MultinomialNB().fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("bayes")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

#試用KNN分群
# from sklearn.neighbors import KNeighborsClassifier
# st=time.time()
clf = KNeighborsClassifier(15).fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("KNN")
pprint.pprint(predicted)
# ed=time.time()
# print(ed-st)

#試用svm分群
from sklearn import svm
clf = svm.SVC(kernel="linear").fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)

pprint.pprint(predicted)
ed=time.time()
print(ed-st)

# 自己肉眼辨識的答案是 合菜 美式 義式 西點 鍋類 冰品 臭豆腐 其他 義式 美式 小吃 泰式 台菜

# 最準的是 Roccio > svm > KNN > bayes，但因為Roccio有些店家無法分類，因此以SVM為準

In [ ]:
#參考PTT斷詞文章
docs_new

In [ ]:
dienallpinlist[0]

In [ ]:
gc.collect()

# 試者用自己的字典給店家分數(一篇評論的正負評價詞最多算5個)

In [33]:
#各店家評論diencommentlist
#取得各店家的斷完詞評論contentcutlist
#自製餐廳相關字字典pinlist
#pindict每個字的評分dict
#各店家評論diencommentlist

#pinCountMaxper5list 一個店家出現的餐廳相關詞數量(一個篇評論的相同詞最多給5個分數)
st=time.time()
pinCountMaxper5list = []
countN=0
for dienallpin in dienallpinlist:
    dienWordcount={}
    for allpin in  dienallpin:
        countN+=1
        if countN%100==0:
            print("已算完"+str(countN)+"篇情緒字")
        for pinword in pinlist:
            if len(re.findall(pinword,allpin))>0:
                if len(re.findall(pinword,allpin))>5:
                    npinword=5
                else:
                    npinword=len(re.findall(pinword,allpin))
                if pinword not in dienWordcount:
                    dienWordcount[pinword]=0
                dienWordcount[pinword]+=npinword
    pinCountMaxper5list.append(dienWordcount)
ed=time.time()
print(ed-st)

已算完100篇情緒字
已算完200篇情緒字
已算完300篇情緒字
已算完400篇情緒字
已算完500篇情緒字
已算完600篇情緒字
已算完700篇情緒字
已算完800篇情緒字
已算完900篇情緒字
已算完1000篇情緒字
已算完1100篇情緒字
已算完1200篇情緒字
已算完1300篇情緒字
已算完1400篇情緒字
已算完1500篇情緒字
已算完1600篇情緒字
已算完1700篇情緒字
已算完1800篇情緒字
已算完1900篇情緒字
已算完2000篇情緒字
已算完2100篇情緒字
已算完2200篇情緒字
已算完2300篇情緒字
已算完2400篇情緒字
已算完2500篇情緒字
已算完2600篇情緒字
已算完2700篇情緒字
已算完2800篇情緒字
已算完2900篇情緒字
已算完3000篇情緒字
已算完3100篇情緒字
已算完3200篇情緒字
已算完3300篇情緒字
已算完3400篇情緒字
已算完3500篇情緒字
已算完3600篇情緒字
已算完3700篇情緒字
已算完3800篇情緒字
已算完3900篇情緒字
已算完4000篇情緒字
已算完4100篇情緒字
已算完4200篇情緒字
已算完4300篇情緒字
已算完4400篇情緒字
已算完4500篇情緒字
已算完4600篇情緒字
已算完4700篇情緒字
已算完4800篇情緒字
已算完4900篇情緒字
已算完5000篇情緒字
已算完5100篇情緒字
已算完5200篇情緒字
已算完5300篇情緒字
已算完5400篇情緒字
已算完5500篇情緒字
已算完5600篇情緒字
已算完5700篇情緒字
已算完5800篇情緒字
已算完5900篇情緒字
已算完6000篇情緒字
已算完6100篇情緒字
已算完6200篇情緒字
已算完6300篇情緒字
已算完6400篇情緒字
已算完6500篇情緒字
已算完6600篇情緒字
已算完6700篇情緒字
已算完6800篇情緒字
已算完6900篇情緒字
已算完7000篇情緒字
已算完7100篇情緒字
已算完7200篇情緒字
已算完7300篇情緒字
已算完7400篇情緒字
已算完7500篇情緒字
已算完7600篇情緒字
已算完7700篇情緒字
已算完7800篇情緒字
已算完7900篇情緒字
已算完8000篇情緒字
已算完8100篇情緒字
已算完8200篇情緒字
已算完8300篇情緒字
已算完8400篇情緒字
已

In [34]:
pinCountMaxper5list[50]

{'': 432,
 'Q彈': 1,
 '一品': 3,
 '一般般': 1,
 '不太喜歡': 2,
 '不好吃': 2,
 '不愛': 4,
 '不膩口': 4,
 '也不錯': 8,
 '乾了點': 1,
 '乾淨': 5,
 '亂': 5,
 '享受': 9,
 '便宜': 10,
 '偏酸': 1,
 '健康': 9,
 '入口即化': 1,
 '划算': 5,
 '剛好': 16,
 '口感紮實': 1,
 '古色古香': 1,
 '可愛': 20,
 '吃飽': 1,
 '合理': 2,
 '喜歡': 45,
 '大份': 2,
 '大碗': 3,
 '天然': 8,
 '天然食材': 2,
 '天龍': 4,
 '太多了': 3,
 '太甜': 14,
 '太臭': 1,
 '好吃': 96,
 '好喝': 4,
 '好好吃': 12,
 '好有味道': 2,
 '好甜': 3,
 '好貴': 3,
 '好食': 1,
 '實惠': 1,
 '寬敞': 1,
 '小巧': 4,
 '小貴': 1,
 '平價': 3,
 '平心而論': 1,
 '很不錯': 3,
 '很便宜': 3,
 '很喜歡': 7,
 '很好吃': 16,
 '很有禮貌': 1,
 '很棒': 1,
 '很清爽': 2,
 '很濃郁': 10,
 '很甜': 1,
 '很美味': 1,
 '很脆': 1,
 '很膩': 1,
 '很難吃': 1,
 '很香': 4,
 '後悔': 2,
 '態度好': 1,
 '整齊': 4,
 '新鮮': 11,
 '明亮': 5,
 '有禮貌': 1,
 '有落差': 1,
 '有點小': 2,
 '有點硬': 3,
 '柔軟': 3,
 '浪漫': 2,
 '清幽': 1,
 '清爽': 16,
 '溫暖': 3,
 '滿喜歡': 1,
 '滿意': 1,
 '漂亮': 4,
 '漲成': 1,
 '濃郁': 42,
 '熱呼呼的': 1,
 '熱情': 6,
 '爽口': 1,
 '特別': 21,
 '甜度恰到好處': 2,
 '精緻': 6,
 '精美': 3,
 '經濟實惠': 1,
 '綿密': 21,
 '綿密口感': 1,
 '綿綿的': 2,
 '缺陷': 1,
 '美味': 16,
 '脆脆的': 3,
 '膩口': 

In [35]:
#店家評論總數Ncommentlist
#店家回覆總數Nmessage
#將六篇回覆視為一篇文章，做正規化(防止評論數過多的店家分數極端化)
Npartpin=[int(Nmessage/6)+Ncomment for Nmessage,Ncomment in zip(Nmessagelist,Ncommentlist)]


#大致計算每家店的分數dienscorelist
dienscorelist=[]
for pinCount,N in zip(pinCountMaxper5list,Npartpin):
    dienscore=0
    for pin in pinCount:
        dienscore+=pinCount[pin]*pindict[pin]/N
    dienscorelist.append(dienscore)

In [36]:
dienscorelist[2]

8.74269662921348

In [37]:
#mydict為自己的字典的資料
#格式為:
######否開頭tag 為 非否tag 反義
#給予正分數 tag評價詞(非否評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予負分數 tag評價詞(否開頭評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予正分數 tag評價詞(非否評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予負分數 tag評價詞(否開頭評價) 同意字群同意字群同意字群同意字群同意字群同意字群

#line.split(",")[1]為tag!!! word為與tag對應的同義字
synonyms=[{word:line.split(",")[1] for word in line.split(",")[1:]} for line in mydict.split("\n") if line.split(",")[0]!=""]

pinsynonymsdict={}
for synonym in synonyms:
    pinsynonymsdict.update(synonym)
del pinsynonymsdict[""]




In [38]:
pinsynonymsdict

{'CP值很低': '否划算',
 'CP值非常高': '划算',
 'CP值高份量夠': '划算',
 'Q彈': '食物美味',
 'Q彈柔嫩': '食物美味',
 'cp值不高': '否划算',
 'cp值太低': '否划算',
 'cp值很高': '划算',
 'cp值高': '划算',
 'hen好吃': '食物美味',
 'juicy': '食物美味',
 '一個人吃剛剛好': '小份量',
 '一口想把它喝光的衝動': '食物美味',
 '一品': '食物美味',
 '一問三不知': '否服務好',
 '一塵不染': '環境好',
 '一定要稱讚一下': '食物美味',
 '一樣無敵': '食物美味',
 '一致好評': '食物美味',
 '一般般': '否食物美味',
 '不乾不柴': '食物美味',
 '不乾柴': '食物美味',
 '不乾淨': '否環境好',
 '不划算': '否划算',
 '不卡牙': '食物美味',
 '不可多得': '服務好',
 '不夠酥脆': '否食物美味',
 '不太優': '否食物美味',
 '不太喜歡': '否食物美味',
 '不太好吃': '否食物美味',
 '不太愛吃': '否食物美味',
 '不太慣': '否食物美味',
 '不好吃': '否食物美味',
 '不對味': '否食物美味',
 '不怎麼樣': '否食物美味',
 '不想離開': '其他正面情緒',
 '不愛': '否食物美味',
 '不愛吃': '否食物美味',
 '不愛它': '否食物美味',
 '不搭': '否食物美味',
 '不敢恭維': '否環境好',
 '不斐': '否划算',
 '不新鮮': '否食物美味',
 '不會再來': '否其他正面情緒',
 '不會去第二次': '否其他正面情緒',
 '不會吃不起': '划算',
 '不會太重': '口味清淡',
 '不會太鹹': '口味清淡',
 '不會過於甜膩': '食物美味',
 '不會過甜': '食物美味',
 '不會過膩': '食物美味',
 '不油膩': '食物美味',
 '不熱': '否食物美味',
 '不用擔心': '其他正面情緒',
 '不用贅述就是好吃阿': '食物美味',
 '不算便宜': '否划算',
 '不脆': '否食物美味',
 '不膩口': '食物美味',


In [39]:
#每篇文章pinCountMaxper5list評價詞出現次數
#評價同義詞pinsynonymsdict

#取得各餐廳的評價特徵分數featurescores!!!
featurescores=[]

#將評價詞加總及評論次數做iterate
for pinCountMaxper5,N in zip(pinCountMaxper5list,Npartpin):
    dienfeaturescore={}
    
    # 評價詞出現次數/評論次數 做為分數，其中否字開頭給負分
    for pin in pinCountMaxper5:
        if pin !="":
            if pinsynonymsdict[pin][0]=="否":
                if pinsynonymsdict[pin][1:] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin][1:]]=0
                dienfeaturescore[pinsynonymsdict[pin][1:]]-=round(pinCountMaxper5[pin]/N,2)*1
            else:
                if pinsynonymsdict[pin] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin]]=0
                dienfeaturescore[pinsynonymsdict[pin]]+=round(pinCountMaxper5[pin]/N,2)
    if len(dienfeaturescore)==0:
        dienfeaturescore["無評"]=1
    featurescores.append(dienfeaturescore)
    
    
    

# 將上述屬性結合成一個大分析表

In [40]:
BigAnalyzeTable=[]
for name,style,Ncomment,contentcut,wordCount,TFIDF,pinCountMax5,score,featurescore,introduction,coordinate in zip(dienlist,stylelist,Npartpin,pincutlist,wordCountlist,TF_IDFlist,pinCountMaxper5list,dienscorelist,featurescores,introductionlist,coordinatelist):
    dien={}
    dien['name']=name
    dien['style']=style
    dien['Ncomment']=Ncomment
    dien['contentcut']=contentcut
    dien['wordCount']=wordCount
    dien['TF_IDF']=TFIDF
    dien['pinCountMaxper5']=pinCountMax5
    dien['score']=score
    dien['featurescores']=featurescore
    dien['tags']=[tag[0] for tag in Counter(TFIDF).most_common(5)]
    dien['introduction']=introduction
    dien['coordinate']=coordinate

    BigAnalyzeTable.append(dien)


# dienlist
# stylelist
# Ncommentlist
# pincutlist
# wordCountlist
# TF_IDFlist
# pinCountMaxper5list
# dienscorelist
# featurescores


#店家種類字典introductionlist
# introductionlist = [dien["introduction"] for dien in ipeenjson]

#店家種類字典coordinatelist
# coordinatelist = [dien['coordinate'] for dien in ipeenjson]

In [41]:
gc.collect()

0

# 將每個自定義的評價詞做正規化評分、並且將分數達標者做為標籤貼上!

In [42]:
#算出每個評論的評論數N時的 最大值b 最小值g for正規化


#allpin=評論詞list
allpin=set([line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"])
standard={}

#standard為正規化所需用的值
for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
        get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
        get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
        standard.update(get)
        print(pin+str(rang[0])+"~"+str(rang[1]-1)+"\t"+str(g)+"\t"+str(b))

重口味0~4	5.0	0
重口味5~9	1.9	0
重口味10~14	1.3	0
重口味15~99	1.3	0
小份量0~4	5.0	0
小份量5~9	2.4	0
小份量10~14	1.7	0
小份量15~99	1.5	0
環境好0~4	26.0	-3.0
環境好5~9	6.7	-0.6
環境好10~14	6.1	-0.5
環境好15~99	5.3	-0.3
偏酸0~4	1.0	0
偏酸5~9	0.5	0
偏酸10~14	0.3	0
偏酸15~99	0.4	0
偏甜0~4	5.0	0
偏甜5~9	1.3	0
偏甜10~14	1.1	0
偏甜15~99	1.7	0
食物很香0~4	8.0	0
食物很香5~9	3.7	0
食物很香10~14	2.3	0
食物很香15~99	4.2	0
食物美味0~4	37.0	-19.0
食物美味5~9	23.0	0
食物美味10~14	19.9	0
食物美味15~99	21.6	0
食物精緻漂亮0~4	15.0	0
食物精緻漂亮5~9	7.2	0
食物精緻漂亮10~14	4.5	0
食物精緻漂亮15~99	6.0	0
其他正面情緒0~4	18.0	-2.0
其他正面情緒5~9	7.5	-0.3
其他正面情緒10~14	9.7	0
其他正面情緒15~99	8.1	0
份量大0~4	9.0	0
份量大5~9	2.7	0
份量大10~14	3.5	0
份量大15~99	3.1	0
偏辣0~4	5.0	0
偏辣5~9	1.3	0
偏辣10~14	0.9	0
偏辣15~99	1.3	0
服務好0~4	3.0	-3.0
服務好5~9	1.9	-0.5
服務好10~14	1.0	-0.2
服務好15~99	0.9	-0.2
口味清淡0~4	3.0	0
口味清淡5~9	0.8	0
口味清淡10~14	0.5	0
口味清淡15~99	1.1	0
划算0~4	14.0	-7.0
划算5~9	5.5	-4.9
划算10~14	5.1	-1.5
划算15~99	2.5	-1.6


In [54]:
xxx='''字義+數量	最高	最低
食物美味0~2	37.1	-21.0
食物美味3~6	21.0	-13.1
食物美味7~13	16.8	-9.2
食物美味14~20	8.6	-4.1
食物美味20~30	5.6	-2.4'''
print(xxx)


字義+數量	最高	最低
食物美味0~2	37.1	-21.0
食物美味3~6	21.0	-13.1
食物美味7~13	16.8	-9.2
食物美味14~20	8.6	-4.1
食物美味20~30	5.6	-2.4


In [58]:
xxx='''字義+數量	最高	最低
食物美味0~2	1.3	-0.7
食物美味3~6	1.2	-0.8
食物美味7~13	1.3	-0.7
食物美味14~20	1.3	-0.7
食物美味20~30	1.4	-0.6'''
print(xxx)

字義+數量	最高	最低
食物美味0~2	1.3	-0.7
食物美味3~6	1.2	-0.8
食物美味7~13	1.3	-0.7
食物美味14~20	1.3	-0.7
食物美味20~30	1.4	-0.6


In [43]:
standard

{'份量大0': (4.5, 4.5),
 '份量大10': (1.75, 1.75),
 '份量大15': (1.55, 1.55),
 '份量大5': (1.35, 1.35),
 '偏甜0': (2.5, 2.5),
 '偏甜10': (0.55, 0.55),
 '偏甜15': (0.85, 0.85),
 '偏甜5': (0.65, 0.65),
 '偏辣0': (2.5, 2.5),
 '偏辣10': (0.45, 0.45),
 '偏辣15': (0.65, 0.65),
 '偏辣5': (0.65, 0.65),
 '偏酸0': (0.5, 0.5),
 '偏酸10': (0.15, 0.15),
 '偏酸15': (0.2, 0.2),
 '偏酸5': (0.25, 0.25),
 '其他正面情緒0': (8.0, 10.0),
 '其他正面情緒10': (4.85, 4.85),
 '其他正面情緒15': (4.05, 4.05),
 '其他正面情緒5': (3.6, 3.9),
 '划算0': (3.5, 10.5),
 '划算10': (1.7999999999999998, 3.3),
 '划算15': (0.44999999999999996, 2.05),
 '划算5': (0.2999999999999998, 5.2),
 '口味清淡0': (1.5, 1.5),
 '口味清淡10': (0.25, 0.25),
 '口味清淡15': (0.55, 0.55),
 '口味清淡5': (0.4, 0.4),
 '小份量0': (2.5, 2.5),
 '小份量10': (0.85, 0.85),
 '小份量15': (0.75, 0.75),
 '小份量5': (1.2, 1.2),
 '服務好0': (0.0, 3.0),
 '服務好10': (0.4, 0.6),
 '服務好15': (0.35, 0.55),
 '服務好5': (0.7, 1.2),
 '環境好0': (11.5, 14.5),
 '環境好10': (2.8, 3.3),
 '環境好15': (2.5, 2.8),
 '環境好5': (3.0500000000000003, 3.65),
 '重口味0': (2.5, 2.5),
 '重口味10': (0.65,

In [44]:
#進行正規化的處理
for dien in BigAnalyzeTable:
    N=dien['Ncomment']
    for feature in dien['featurescores']:
        if feature !="無評":
            if N<6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(0)][1],2)
            elif N<11 and N>=6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(5)][1],2)
            elif N<16 and N>=11:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(10)][1],2)
            else:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(15)][1],2)

# 正規化後，每個屬性的評價區間都為2! 但未避免有些沒被負評的店判斷成負評，不將中間值移至0的位置

In [45]:


allpin=[line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"]
standard={}

for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
#         get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
#         get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
#         standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

食物美味+0	1.3	-0.7
食物美味+5	2.0	0
食物美味+10	2.0	0
食物美味+15	2.0	0
食物美味+0	1.3	-0.7
食物美味+5	2.0	0
食物美味+10	2.0	0
食物美味+15	2.0	0
划算+0	1.3	-0.7
划算+5	1.1	-0.9
划算+10	1.5	-0.4
划算+15	1.2	-0.8
環境好+0	1.8	-0.2
環境好+5	1.8	-0.1
環境好+10	1.9	-0.1
環境好+15	1.9	-0.1
服務好+0	1.0	-1.0
服務好+5	1.6	-0.4
服務好+10	1.6	-0.4
服務好+15	1.6	-0.3
食物精緻漂亮+0	2.0	0
食物精緻漂亮+5	2.0	0
食物精緻漂亮+10	2.0	0
食物精緻漂亮+15	2.0	0
食物很香+0	2.0	0
食物很香+5	2.0	0
食物很香+10	2.0	0
食物很香+15	2.0	0
其他正面情緒+0	1.8	-0.2
其他正面情緒+5	1.9	-0.1
其他正面情緒+10	2.0	0
其他正面情緒+15	2.0	0
份量大+0	2.0	0
份量大+5	2.0	0
份量大+10	2.0	0
份量大+15	2.0	0
小份量+0	2.0	0
小份量+5	2.0	0
小份量+10	2.0	0
小份量+15	2.0	0
重口味+0	2.0	0
重口味+5	2.0	0
重口味+10	2.0	0
重口味+15	1.9	0
口味清淡+0	2.0	0
口味清淡+5	2.1	0
口味清淡+10	2.2	0
口味清淡+15	2.1	0
偏辣+0	2.0	0
偏辣+5	2.0	0
偏辣+10	1.9	0
偏辣+15	1.9	0
偏甜+0	2.0	0
偏甜+5	2.1	0
偏甜+10	2.0	0
偏甜+15	2.0	0
偏酸+0	2.0	0
偏酸+5	2.0	0
偏酸+10	2.1	0
偏酸+15	1.9	0


# 貼餐廳評價標籤!

In [ ]:
pipin=[('食物美味','食物不好吃',"非常好吃",'非常不好吃'),('划算','價格較高昂',"非常便宜",'非常貴')
       ,('環境好','環境比較不好',"環境超棒",'環境很差'),('服務好','服務較差',"服務一級棒",'服務很差')
       ,('其他正面情緒','評論有些負面情緒詞',"網路評論正面情緒非常多",'網路評論負面情緒很多')]
moods=[mood[0] for mood in pipin]
for dien in BigAnalyzeTable:
    newtag=[]
    for feature in dien["featurescores"]:
        if feature in moods:
            if dien["featurescores"][feature]>1:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[2])
            elif dien["featurescores"][feature]>0.4:
                newtag.append(feature)
            elif dien["featurescores"][feature]<-0.5:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[3])
            elif dien["featurescores"][feature]<0.0:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[1])
        else:
            if dien["featurescores"][feature]>0.7:
                newtag.append(feature)
                    
                    
#用set去除重複標籤
    dien['tags']=list(set(dien['tags']+newtag))
    print(dien['name'])
#     print(newtag)
    dien['tags'].append(dien['style'])
    print(dien['tags'])

In [ ]:
for i in BigAnalyzeTable:
    if i['score']>30 and i['Ncomment']>10:
        print(i['name']+' '+str(i['score']))
#         print(i['tags'])
        print(i['featurescores'])
        print('\n')
gooddien=[i for i in BigAnalyzeTable if i['score']>30 and i['Ncomment']>10]
# len(gooddien)

In [ ]:
with open('D:/Data/JsonData/TainanFood/bigtable_1.3.json','w') as f:
    json.dump(BigAnalyzeTable,f)

In [1]:
import json
with open('D:/Data/JsonData/TainanFood/bigtable_1.3.json') as f:
    data=json.load(f)

In [2]:
[dien['coordinate'] for dien in data]

[{'lat': 22.9999252, 'lng': 120.1534308},
 {'lat': 22.9841793, 'lng': 120.2185374},
 {'lat': 22.9903124, 'lng': 120.1981439},
 {'lat': 22.9973997, 'lng': 120.1991831},
 {'lat': 22.9980996, 'lng': 120.1741423},
 {'lat': 22.989468, 'lng': 120.162831},
 {'lat': 22.9910369, 'lng': 120.2214746},
 {'lat': 22.9890556, 'lng': 120.2041456},
 {'lat': 23.0004936, 'lng': 120.2068603},
 {'lat': 22.9891486, 'lng': 120.1951976},
 {'lat': 22.9945217, 'lng': 120.1971865},
 {'lat': 22.994548, 'lng': 120.2133713},
 {'lat': 22.9976135, 'lng': 120.1989103},
 {'lat': 23.0518138, 'lng': 120.1646187},
 {'lat': 22.9942969, 'lng': 120.2001762},
 {'lat': 22.973827, 'lng': 120.220684},
 {'lat': 22.9982344, 'lng': 120.174619},
 {'lat': 22.9933486, 'lng': 120.1976249},
 {'lat': 23.02912, 'lng': 120.2518763},
 {'lat': 22.9889141, 'lng': 120.1880278},
 {'lat': 22.9981748, 'lng': 120.1743296},
 {'lat': 22.9964763, 'lng': 120.2030135},
 {'lat': 22.9892031, 'lng': 120.1949238},
 {'lat': 22.9987503, 'lng': 120.1696348},


In [ ]:
from collections import Counter
Counter(dict(data[0]['wordCount'])).most_common(50)

In [ ]:
dict(data[0]['wordCount'])

In [9]:
for dien in data[:10]:
    print(dien['name'])
    print(round(dien['score'],2))
    print(dien['featurescores'])
    

同記安平豆花(安平總店)
6.22
{'食物美味': 0.37, '食物很香': 0.22, '食物精緻漂亮': 0.19, '其他正面情緒': 0.19, '環境好': 0.14, '份量大': 0.13, '偏甜': 0.24, '划算': 0.19, '口味清淡': 0.07, '偏酸': 0.2, '服務好': -0.15}
深藍咖啡館
8.77
{'食物美味': 0.61, '划算': -0.8, '環境好': 0.25, '食物很香': 0.29, '其他正面情緒': 0.21, '偏甜': 0.33, '食物精緻漂亮': 0.3, '份量大': 0.04, '小份量': 0.05, '服務好': 0.13, '重口味': 0.06}
阿堂鹹粥
8.74
{'食物美味': 0.61, '食物很香': 0.1, '划算': -0.23, '環境好': 0.04, '份量大': 0.48, '食物精緻漂亮': 0.23, '其他正面情緒': 0.24, '重口味': 0.08, '口味清淡': 0.02, '偏甜': 0.12, '服務好': 0.09, '偏辣': 0.02}
富盛號碗粿
11.17
{'食物美味': 0.58, '食物精緻漂亮': 0.24, '食物很香': 0.12, '划算': 0.18, '環境好': 0.29, '其他正面情緒': 0.24, '重口味': 0.14, '偏辣': 0.09, '口味清淡': 0.04, '偏甜': 0.41, '份量大': 0.4, '服務好': 0.11, '小份量': 0.03}
依蕾特布丁奶酪(安平門市)
9.08
{'食物美味': 0.55, '食物很香': 0.29, '食物精緻漂亮': 0.1, '其他正面情緒': 0.15, '環境好': 0.08, '份量大': 0.08, '划算': 0.06, '偏甜': 0.29, '口味清淡': 0.07, '偏酸': 0.2, '小份量': 0.05}
食下有約 想法廚房
34.61
{'食物美味': 1.36, '食物很香': 0.82, '環境好': 0.97, '食物精緻漂亮': 0.97, '份量大': 0.54, '其他正面情緒': 0.66, '划算': 0.18, '口味清淡': 0.07, '偏甜': 0.28, '服務好

In [6]:
data[0].keys()

dict_keys(['name', 'style', 'Ncomment', 'contentcut', 'wordCount', 'TF_IDF', 'pinCountMaxper5', 'score', 'featurescores', 'tags', 'introduction', 'coordinate'])